# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

## Note:
### I decided to use transcripys of Star Trek Voyager as I have never watched Seinfeld and I thought I would find something I know well more enjoyable. I hope this is okay :)

In [1]:
import signal
import requests
from torch.autograd import Variable
from contextlib import contextmanager
import re
from collections import Counter


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler


In [2]:
from torch.optim import lr_scheduler

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './voyager_transcripts_all_cleaned.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [5]:
view_line_range = (30, 40)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 49524
Number of lines: 102509
Average number of words in each line: 9.741603176306471

The lines 30 to 40:
CHAKOTAY: That's where I'm going. 
TUVOK: Plasma storm density increasing by fourteen percent. Twenty. Twenty five. 
CHAKOTAY: Hold on!
(The little ship dodges around the plasma streamers.) 
TUVOK: The Cardassian ship is not reducing power. They're following us in.
CHAKOTAY: Gul Evek must feel daring today.
TUVOK: They've taken a hit on their port blade. They're sending out a distress signal on all Cardassian frequencies.
CHAKOTAY: Can you plot a course through these plasma fields, Mister Tuvok?
TUVOK: The storm's activity is typically widespread in this vicinity. I can plot a course, but I'm afraid it will require an indirect route.
CHAKOTAY: We can use a time to make some repairs. What was that?


---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [15]:
import problem_unittests as tests

In [16]:

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: Two dictionaries, vocab_to_int, int_to_vocab
    """
    text_counts = Counter(text)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(text_counts, key=text_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: text for ii, text in enumerate(sorted_vocab)}
    vocab_to_int = {text: ii for ii, text in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [18]:
def old_token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punct_dict={".":"||Fullstop||",
           ",":"||Comma||",
           "!":"||ExclamationMark||",
           ";":"||SemiColon||",
           "\"":"||QuotationMark||",
           "?":"||QuestionMark||",
           "(":"||LeftParentheses||",
           ")":"||RightParentheses||",
           "-":"||Dash||",
           "\n":"||Return||"}
    
        
    return punct_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(old_token_lookup)

Tests Passed


## Note:
I needed a different token lookup for my scripts, it didnt pass your test and returned the error Unknown symbols: {'<', '>', ']', '['} So I have demonstrated above that I can make a def that can pass your tests I just wont be using it due to the different nature of my scripts :)

In [19]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punct_dict={".":"||Fullstop||",
           ",":"||Comma||",
           "!":"||ExclamationMark||",
           ";":"||SemiColon||",
           "\"":"||QuotationMark||",
           "?":"||QuestionMark||",
           "(":"||LeftParentheses||",
           ")":"||RightParentheses||",
           "-":"||Dash||",
           "\n":"||Return||",
            "<":"||LessThan||",
            ">":"||LessThan||",
            "[":"||SquareOpen||",
            "]":"||SquareClose||"}
    
        
    return punct_dict

## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
#import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
(I changed a little bit, I like having functions that return statements for both cases)
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print('GPU found!')
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

GPU found!


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [24]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    n_batches = len(words)//batch_size
    # only full batches
    words = words[:n_batches*batch_size]
    y_len = len(words) - sequence_length
    x, y = [], []
    for idx in range(0, y_len):
        end_point = sequence_length + idx
        x_batch = words[idx:end_point]
        x.append(x_batch)
        batch_y =  words[end_point]
        y.append(batch_y)    

    # create Tensor datasets, int64 is trying to avoid issues when running on windows
    data = TensorDataset(torch.from_numpy(np.asarray(x, dtype='int64')), torch.from_numpy(np.asarray(y, dtype='int64')))
    # make sure the SHUFFLE your training data
    data_loader = DataLoader(data, batch_size=batch_size)
    # return a dataloader
    return data_loader  


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [25]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]])

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [31]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5,lr=0.001):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # define embedding layer        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        ## Define the LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # Define the final, fully-connected output layer
        self.fc = nn.Linear(hidden_dim, output_size)

        
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """

        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.fc(lstm_out)
        
        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
        # last batch
        out = out[:, -1]

        return out, hidden

    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [33]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden, clip=9):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()

    hidden1 = tuple([each.data for each in hidden])
    rnn.zero_grad()
    optimizer.zero_grad()
    
    try:
        output, hidden1 = rnn(inp, hidden1)        
    except RuntimeError:
        raise
    loss = criterion(output, target)
    loss.backward()
    
    # clip_grad_norm helps prevent exploding gradient
    nn.utils.clip_grad_norm_(rnn.parameters(),  clip)
    optimizer.step()

    return loss.item(), hidden1

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
(I have modified it so that it saves every time the loss has decreased at the end of an epoch, 
inaase the last step jumps up for some reason)
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, 
              show_every_n_batches, save_path):
    batch_losses = []
    rnn.cuda()
    rnn.train()
    
    valid_loss_min = np.Inf
    last_average = np.Inf

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        #print('epoch:',epoch_i)
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs,
                                             labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
            
            last_average= np.average(batch_losses)
            batch_losses = []
                
                #saving model if loss has decreased
        print('epoch:',epoch_i,'complete , loss:',last_average)
        if (last_average <= valid_loss_min):
            print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format
              (valid_loss_min, last_average))
            helper.save_model(save_path, trained_rnn)
            valid_loss_min = last_average
                                
                
        scheduler.step()
        

    # returns a trained rnn
    return rnn



### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [35]:
# Data params
# Sequence Length
sequence_length =  12 # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [38]:
# Training parameters
# Number of Epochs
num_epochs = 60
# Learning Rate
learning_rate = 0.003

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 250
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 2000



### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [39]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
I have changed it slightly, just to account for the saving when runnning :)
"""


# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.3)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.98)

# training the model

trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, 
                        num_epochs, show_every_n_batches, './trained_voy17backup')

# saving the trained model
helper.save_model('./save/trained_voy17final', trained_rnn)
print('Model Trained and Saved')

#voy4 is fully trained on three s's
#voy8backup is fully trained on all s's
#12 only has 2 epochs but actually makes stuff!
#13 on all s's and 35 epochs
#15 on all s's cleaned and 60 epochs

Training for 1 epoch(s)...
Epoch:    1/1     Loss: 6.824861526489258

Epoch:    1/1     Loss: 6.361799716949463

Epoch:    1/1     Loss: 6.493287563323975

Epoch:    1/1     Loss: 7.304834842681885

Epoch:    1/1     Loss: 6.567425727844238

epoch: 1 complete , loss: 5.378281593322754
Validation loss decreased (inf --> 5.378282). Saving model...
Model Trained and Saved


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

#Sequence Length
sequence_length =  12 
I found that this gave suffcient level of context without having to process to much data, I started with 8 and found that after I retrained with 12 it gave better grammar.

#Batch Size
batch_size = 128
To be honest I remembered from the video to chose either 32, 64 or 128 and seeing as we have a nice gpu to use I went for 128

#Decaying Learning Rate
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.98)
Looking at the pytorch documentation this was the simplest way to impliment a decaying learning rate. Given that I started at 0.003, to end up with 0.003*0.98^60 = 0.0009 seemed like a good way to home in on the minima

#Number of Epochs
num_epochs = 30
Interestingly I did try and train for 60 epochs and it didnt seem to improve much more after 45.

#Learning Rate
learning_rate = 0.003
I tried with 0.01 and the learning was jumping all over the place, and with 0.001 it seemed to converge too slowly to start, hence the 0.003 to start with decay over epochs.

#Embedding Dimension
embedding_dim = 200
This seemed like a standard value to use.

#Hidden Dimension
hidden_dim = 250
#Number of RNN Layers
n_layers = 3

I felt these values for the hidden_dim and n_layers would give sufficient complexity whilst not making the cimputation time explode too quickly. I did try hidden_dim=200 at the start but changed it at the same time as the sequence length to try and improve grammar.


---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [66]:

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
#import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_voy15final')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [67]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    #
    
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
               
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
           # print('moving to cpu')
            p = p.cpu() # move to cpu
            
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq=current_seq.cpu() #needed to prevent error
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### I wanted a function where you could input a sentance not just a word

In [68]:
import torch.nn.functional as F

def generate_from_sentance(rnn, sentance, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
   
    #prime_id
    rnn.eval()
    
    predicted= []
    #turn my sentance into numbers
    my_text = sentance

    for key, token in token_dict.items():
        my_text = my_text.replace(key, ' {} '.format(token))

    my_text = my_text.lower()
    my_text = my_text.split()

    int_text = [vocab_to_int[word] for word in my_text]
    
    #make the sequence
    current_seq = np.full((1, sequence_length), pad_value)
    
    #for each number in sentance roll the sequence and add it, add the word to predicted

    for i in range(len(int_text)):
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = int_text[i]
        word = int_to_vocab[int_text[i]]
        predicted.append(word)  
    print('your input is:', predicted)
    print(' ')

    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
               
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
           # print('moving to cpu')
            p = p.cpu() # move to cpu
            
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq=current_seq.cpu() #needed to prevent error
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
        

    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [74]:
gen_length = 100
sentance = 'the EPS manifold' #sentance for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
SPECIAL_WORDS = {'PADDING': '<PAD>'}
pad_word = helper.SPECIAL_WORDS['PADDING']

for _ in range(5):
    generated_script = generate_from_sentance(trained_rnn, sentance, int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
    print(generated_script)
    print(' ')
    print('---------------')
    print(' ')

your input is: ['the', 'eps', 'manifold']
 
the eps manifold. the ship exhibits breaching.)
tuvok: shields at sixty percent. shields are down, and the port nacelle is collapsing.)
tuvok: captain!
janeway: janeway to sickbay.
torres[ oc] : go ahead, captain.
janeway: janeway to chakotay.
emh[ on monitor] : you may have drunk the situation, mister vulcan.

[ sickbay]

(the emh brings her bouquet the doors.)
seven: the rift is outmanoeuvring.
chakotay: i'm sorry.

 
---------------
 
your input is: ['the', 'eps', 'manifold']
 
the eps manifold.
janeway: i understand how to convince you to stay home. i want to see you to the gambling medical procedure.
emh: you must be implying you have a son.
admiral: oh, i don't have a forensic bedside to a romantic involvement. executing a false case.
surgeon: what do you propose?
security: i'm not sure that i had some annoying ideas.
kim: i'm not going to be here. but i haven't seen the shuttle's computers, but i'm not going to decompile the spoils.
 


In [53]:
# run the cell multiple times to get different results!
gen_length = 600 # modify the length to your preference
prime_word = 'torres:' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

torres: the core's data is in communication.
janeway: forcefield destroy.

[ bridge]

chakotay: i can know.

[ who]
(neelix that's been preserve with a consistent one audio.)
seven: you have a achieved harmless of destabilising.
for you have to during me, death. i can want to this a containers came, but the core's few days.
seven: the doctor is a prepare preparing of the death. i talk the doctor is in anyway of destabilising.
(the goth is draw.)
janeway: you know what happened, seven. i can't get a assimilation.

[ hope lab]

seven: the burn enjoyed with the burn trouble on the data.
(the evidence grow with the there's.)
seven: i'm lead a vicinity step, soup fight fight fight :. i can't get the order to the priority and take the warp aye to the they'll. we they're use a vessel as we'd as the captain actions.

[ corridor]

(the intend of the beating one audio of units.)
seven: what?
thousand i believe you our protocol it.
seven: if i two codes you, i'm sure you will have to during you.


#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [ ]:
# save script to a text file
f =  open("generated_voy_script_4.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.